# Export Hospital Level Dashboards

**Purpose:**

1. **Optionally authenticate** with `gcloud` (via user prompt).
2. **Query BigQuery** for facility names (`faci_name` from `drg-viz.00_datasets.hci`).
3. **(Optional) Save** these names to a local CSV for debugging.
4. **Launch Selenium** (using `a_source_chrome_webdriver.py`) to open Power BI in **headless** mode,  
   select each facility in the slicer, and **export** a PDF to `./hospital-reports`.


In [ ]:
# Install required packages every run (Jupyter-friendly)
%pip install -r ../requirements.txt

import os
import sys
import csv
import math
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed

sys.path.append(os.path.abspath(".."))
from src.functions import *

HOSPITALS_CSV = os.path.join("..", "data", "inputs", "hospitals.csv")
NUM_WORKERS_NORMAL = 16
TO_DEBUG = False

try:
    with open(HOSPITALS_CSV, newline="") as f:
        hospitals = [row[0] for row in csv.reader(f) if row]
    if hospitals and hospitals[0].strip().lower() == "facility_name":
        hospitals = hospitals[1:]
except Exception as e:
    sys.exit(f"Error reading hospitals CSV: {e}")

# Generate timestamp once for the entire script run
run_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Output folder uses consistent timestamp
output_dir = os.path.join("..", "data", "outputs", f"SB_Report_{run_timestamp}")
os.makedirs(output_dir, exist_ok=True)

if TO_DEBUG:
    failed = worker_task(hospitals[:1], output_dir, 1, run_timestamp)  # Pass timestamp
else:
    try:
        num = input("How many hospitals? ('all' or number): ").strip()
        num_hosp = len(hospitals) if num.lower() == "all" else int(num)
    except ValueError:
        num_hosp = len(hospitals)

    subset = hospitals[:num_hosp]
    try:
        workers = int(input("Number of workers: ").strip())
    except ValueError:
        workers = NUM_WORKERS_NORMAL

    split_size = math.ceil(len(subset) / workers)
    subsets = [subset[i:i+split_size] for i in range(0, len(subset), split_size)]

    failed = []
    with ProcessPoolExecutor(max_workers=workers) as executor:
        futures = {
            executor.submit(worker_task, s, output_dir, i+1, run_timestamp): s
            for i, s in enumerate(subsets)
        }
        for future in as_completed(futures):
            failed.extend(future.result())

if failed:
    fail_csv = os.path.join(output_dir, "failed_hospitals.csv")
    with open(fail_csv, "w", newline="") as f:
        csv.writer(f).writerows([[h] for h in failed])
    print(f"Failed hospitals saved to {fail_csv}")




[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
[Worker 1] Starting with 118 hospital(s).
[Worker 4] Starting with 118 hospital(s).
[Worker 2] Starting with 118 hospital(s).
[Worker 5] Starting with 118 hospital(s).
[Worker 11] Starting with 118 hospital(s).
[Worker 9] Starting with 118 hospital(s).
[Worker 3] Starting with 118 hospital(s).
[Worker 12] Starting with 118 hospital(s).
[Worker 7] Starting with 118 hospital(s).
[Worker 6] Starting with 118 hospital(s).
[Worker 8] Starting with 118 hospital(s).
[Worker 10] Starting with 118 hospital(s).
[Worker 14] Starting with 118 hospital(s).
[Worker 16] Starting with 114 hospital(s).
[Worker 15] Starting with 118 hospital(s).
[Worker 13] Starting with 118 hospital(s).
[Worker 15] No iframe detected.
[Worker 12] No iframe detected.
[Worker 12] Failed for PANTABANGAN INFIRMARY: Message: no such element: Unable